# CUTLAB AI – Next-Gen AI Video Editor

## 1. Problem Definition & Objective

### Selected Project Track
**AI-Powered Creative Tools**

### Clear Problem Statement
Video editing is a time-consuming, labor-intensive process. Creators spend hours scrubbing through raw footage to find the best moments, removing silence, and adjusting framing for different platforms (e.g., landscape for YouTube, portrait for TikTok). This manual workflow is a bottleneck for content creation.

### Objectives of CUTLAB AI
**CUTLAB AI** aims to democratize professional video editing by automating the most tedious parts of the process. Our goal is to build an "AI Co-pilot" that:
- **Detects Scenes**: Automatically segments raw footage into meaningful clips.
- **Understands Content**: Identifies faces, motion, and audio energy.
- **Suggests Edits**: Proposes smart cuts to remove silence or highlight action.
- **Adapts Formats**: Intelligently reframes content for vertical video.

### Real-World Relevance & Motivation
With the explosion of the creator economy, the demand for high-quality, frequent video content is higher than ever. Tools that reduce editing time from hours to minutes have immense value for YouTubers, marketers, and educators.

## 2. Data Understanding & Preparation

### Video as Data
In this system, our "data" consists of raw video files (MP4, MOV). A video is a rich multimodal data source containing:
- **Visuals**: A sequence of image frames (RGB pixel data).
- **Audio**: Waveforms containing speech, music, and background noise.
- **Time**: The temporal dimension that links visuals and audio.

### Preparation Pipeline
Before analysis, raw video goes through a preprocessing pipeline:
1. **Ingestion**: Video is uploaded and metadata (duration, FPS, resolution) is extracted.
2. **Frame Sampling**: We don't process every single frame for expensive tasks. We sample frames at intervals (e.g., every 5th frame) to balance speed and accuracy.
3. **Normalization**: Frames are converted to RGB standard formats for CV models.
4. **Audio Extraction**: Audio tracks are separated for waveform analysis and speech-to-text processing.

## 3. Model / System Design

### Architecture Overview
The system is built as a pipeline of specialized AI modules:

1. **Scene Detection Engine**: 
   - **Technique**: Content-Aware Detection.
   - **Library**: `PySceneDetect` or custom OpenCV logic.
   - **Role**: Breaks video into logical shots based on visual changes.

2. **Smart Human Analysis (Computer Vision)**:
   - **Technique**: Pose Estimation and Face Detection.
   - **Library**: `MediaPipe`.
   - **Role**: Tracks subjects to enable "Smart Crop" and "Face Focus" effects.

3. **Cut Suggestion Logic (Heuristic AI)**:
   - **Technique**: Rule-based scoring system.
   - **Role**: Evaluates each scene based on motion intensity, audio energy, and face presence to suggest "Keep" or "Cut".

### Design Choices
- **OpenCV & MediaPipe**: Chosen for their real-time performance on CPU, allowing the app to run locally without expensive GPU clusters.
- **FastAPI Backend**: Provides a robust, async interface for the frontend to request analysis tasks.

## 4. Core Implementation

Below are the core Python implementations for the AI modules. These snippets represent the actual logic used in the CUTLAB AI backend.

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from typing import List, Dict, Any

# Note: In a real environment, you would run 'pip install mediapipe opencv-python'

def _compute_motion_score(prev_frame: np.ndarray, cur_frame: np.ndarray) -> float:
    """
    Computes motion intensity between two frames using absolute pixel difference.
    Returns a normalized score [0, 1].
    """
    if prev_frame is None:
        return 0.0
    
    # Convert to grayscale for simpler difference calculation
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    cur_gray = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)
    
    diff = cv2.absdiff(prev_gray, cur_gray)
    
    # Normalize by max possible difference (255 * pixels)
    max_diff = 255 * diff.size
    score = diff.sum() / max_diff
    return float(score)

### Human & Face Analysis Module
This module processes video segments to detect faces and calculate motion. This is critical for the "Auto Reframe" feature.

In [ ]:
def analyze_video_segment(video_path: str, start_sec: float, duration: float) -> Dict[str, Any]:
    """
    Analyzes a specific video segment for human presence and motion.
    """
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_frame = int(start_sec * fps)
    end_frame = int((start_sec + duration) * fps)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    # Initialize MediaPipe Face Detection
    mp_face_detection = mp.solutions.face_detection
    face_detector = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)
    
    motion_scores = []
    face_detected_frames = 0
    total_processed = 0
    last_frame = None
    
    while cap.isOpened() and cap.get(cv2.CAP_PROP_POS_FRAMES) < end_frame:
        ret, frame = cap.read()
        if not ret:
            break
            
        total_processed += 1
        
        # 1. Face Detection
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detector.process(rgb_frame)
        if results.detections:
            face_detected_frames += 1
            
        # 2. Motion Analysis
        if last_frame is not None:
            score = _compute_motion_score(last_frame, frame)
            motion_scores.append(score)
            
        last_frame = frame.copy()
        
    cap.release()
    
    avg_motion = sum(motion_scores) / len(motion_scores) if motion_scores else 0.0
    face_presence_ratio = face_detected_frames / total_processed if total_processed > 0 else 0.0
    
    return {
        "timestamp": start_sec,
        "duration": duration,
        "avg_motion": round(avg_motion, 4),
        "has_face": face_presence_ratio > 0.5,
        "face_confidence": round(face_presence_ratio, 2)
    }

### Cut Suggestion Logic
This function represents the decision-making brain of the editor. It takes the raw metrics from the analysis and decides what to do with the clip.

In [ ]:
def generate_suggestion(scene_metrics: Dict[str, Any]) -> Dict[str, Any]:
    """
    Decides whether a scene should be kept, cut, or highlighted based on metrics.
    """
    motion = scene_metrics['avg_motion']
    has_face = scene_metrics['has_face']
    audio_energy = scene_metrics.get('audio_energy', 0.5)  # Placeholder for audio metric
    
    # Rule 1: Static boring scenes (low motion, no face)
    if motion < 0.01 and not has_face:
        return {
            "action": "CUT",
            "reason": "Static scene with no subjects",
            "confidence": 0.9
        }
        
    # Rule 2: High action scenes
    if motion > 0.15:
        return {
            "action": "HIGHLIGHT",
            "reason": "High motion/action detected",
            "confidence": 0.85
        }
        
    # Rule 3: Talking head (Face + moderate motion)
    if has_face:
        return {
            "action": "KEEP",
            "reason": "Subject detected on screen",
            "confidence": 0.95
        }
        
    return {"action": "KEEP", "reason": "Standard scene", "confidence": 0.5}

## 5. Evaluation & Analysis

### Sample Output
If we run the above pipeline on a 10-second clip of a vlogger walking and then sitting down, the system outputs:

```json
{
  "timestamp": 0.0,
  "duration": 5.0,
  "avg_motion": 0.21,
  "has_face": true,
  "suggestion": {
    "action": "HIGHLIGHT",
    "reason": "High motion/action detected"
  }
}
```

### Evaluation Metrics
We evaluate the system performance based on:
1. **Detection Accuracy**: How often are faces correctly identified? (Measured vs. manual ground truth).
2. **Processing Speed**: Real-time factor (RTF). The target is < 0.5x realtime (e.g., 1 min video takes < 30s to process).
3. **User Acceptance**: Percentage of AI suggestions accepted by the user without modification.

## 6. Ethical Considerations & Responsible AI

### Privacy by Design
CUTLAB AI processes videos locally or on secure instances. No video data is used to train global models without explicit user consent. This ensures that personal vlogs or sensitive business footage remains private.

### Bias Mitigation
Face detection models (like MediaPipe) can sometimes exhibit bias across different skin tones or lighting conditions. We mitigate this by:
- Using high-confidence thresholds to avoid false positives.
- Allowing users to manually override any AI decision easily.
- Continuously testing against diverse datasets.

## 7. Conclusion & Future Scope

### Summary
This notebook demonstrated the core AI logic behind CUTLAB AI. By combining Computer Vision (MediaPipe) with heuristic logic, we can successfully automate the repetitive parts of video editing, allowing creators to focus on storytelling.

### Future Improvements
- **Generative AI**: Integrating LLMs to summarize video content and generate titles/descriptions automatically.
- **Voice Commands**: Editing video by speaking to the AI (e.g., "Remove all the silent parts").
- **Style Transfer**: Using GANs to apply cinematic color grading automatically.